## Task1

-> Objetivo: Dada uma consulta especificada pelo usuário e uma lista de produtos correspondentes, o objetivo desta tarefa é classificar os produtos para que os produtos relevantes sejam classificados acima dos não relevantes.

A entrada para esta tarefa será uma lista de consultas com seus identificadores. O sistema terá que gerar um arquivo CSV onde o **query_id** estará na primeira coluna e o **product_id** na segunda coluna, onde para cada **query_id** primeira linha será o produto mais relevante e a última linha o produto menos relevante

Proximos Passos 

1 Criar seçao de metricas

2 criar modelo 

In [2]:
!pip install lightgbm
!pip install annoy
!pip install faiss-cpu
!pip install -U sentence-transformers
!pip install nltk

Requirement already up-to-date: sentence-transformers in /home/erick.correia/.local/lib/python3.8/site-packages (2.2.2)


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import lightgbm as lgb
import matplotlib.pylab as plt
from sklearn.preprocessing import LabelEncoder 
#metricas
from sklearn.metrics import ndcg_score

#modelos
import lightgbm as lgb
import faiss
from annoy import AnnoyIndex

In [22]:
train_df = pd.read_csv('dataset/train-v0.3.csv')
product = pd.read_csv('dataset/product_catalogue-v0.3.csv')

In [23]:
print("shape train data = " , train_df.shape)
print("shape train product = " , product.shape)

shape train data =  (781744, 5)
shape train product =  (883868, 7)


In [24]:
product.info()
product.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883868 entries, 0 to 883867
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   product_id            883868 non-null  object
 1   product_title         883719 non-null  object
 2   product_description   446710 non-null  object
 3   product_bullet_point  739196 non-null  object
 4   product_brand         807927 non-null  object
 5   product_color_name    533660 non-null  object
 6   product_locale        883868 non-null  object
dtypes: object(7)
memory usage: 47.2+ MB


,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpene...",NaN,144 woodcase #2 HB pencils made from high-qual...,Amazon Basics,Yellow,us
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,...",<p><strong>BACK TO BAZIC</strong></p><p>Our go...,&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each...,BAZIC Products,12-count,us
2,B07G7F6JZ6,Emraw Pre Sharpened Round Primary Size No 2 Ju...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 8 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us
3,B07JZJLHCF,Emraw Pre Sharpened Triangular Primary Size No...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 6 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us
4,B07MGKC3DD,"BIC Evolution Cased Pencil, #2 Lead, Gray Barr...",NaN,Premium #2 HB lead pencils with break-resistan...,Design House,Gray,us


In [25]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781744 entries, 0 to 781743
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   query_id      781744 non-null  int64 
 1   query         781744 non-null  object
 2   query_locale  781744 non-null  object
 3   product_id    781744 non-null  object
 4   esci_label    781744 non-null  object
dtypes: int64(1), object(4)
memory usage: 29.8+ MB


,query_id,query,query_locale,product_id,esci_label
0,0,# 2 pencils not sharpened,us,B0000AQO0O,exact
1,0,# 2 pencils not sharpened,us,B0002LCZV4,exact
2,0,# 2 pencils not sharpened,us,B00125Q75Y,exact
3,0,# 2 pencils not sharpened,us,B001AZ1D3C,exact
4,0,# 2 pencils not sharpened,us,B001B097KC,exact


In [26]:
train_df.isnull().sum()

query_id        0
query           0
query_locale    0
product_id      0
esci_label      0
dtype: int64

In [27]:
product.isnull().sum()

product_id                   0
product_title              149
product_description     437158
product_bullet_point    144672
product_brand            75941
product_color_name      350208
product_locale               0
dtype: int64

-> separar o produtos com o locale = us

-> colocar o product_title no train dataset de treino

-> fazer o tratamendo na coluna de title


In [28]:
full_train = pd.merge(train_df, 
                          product[["product_id", "product_locale", "product_title"]],
                          left_on=["product_id", "query_locale"], 
                          right_on=["product_id", "product_locale"]
)


In [29]:
print(full_train.shape)
full_train.head()

(781744, 7)


,query_id,query,query_locale,product_id,esci_label,product_locale,product_title
0,0,# 2 pencils not sharpened,us,B0000AQO0O,exact,us,"Ticonderoga Beginner Pencils, Wood-Cased #2 HB..."
1,8799,pencils for kindergarteners,us,B0000AQO0O,exact,us,"Ticonderoga Beginner Pencils, Wood-Cased #2 HB..."
2,14844,#2 dixon oriole pencils not sharpened,us,B0000AQO0O,substitute,us,"Ticonderoga Beginner Pencils, Wood-Cased #2 HB..."
3,15768,#2 pencils with erasers sharpened not soft,us,B0000AQO0O,substitute,us,"Ticonderoga Beginner Pencils, Wood-Cased #2 HB..."
4,16972,classroom friendly supplies pencil sharpener,us,B0000AQO0O,irrelevant,us,"Ticonderoga Beginner Pencils, Wood-Cased #2 HB..."


**Removendo StopWords / lowcase / caracteres especiais** 

In [8]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [9]:
text = train_df['query']
text = text[:419730]

stop_words = stopwords.words('english')

In [10]:
#print(text)
#text_tokens = word_tokenize(text)
text= text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#remove lowcase
text = text.str.lower()
#revome caracteres especiais 
text = text.str.replace('[#,@,&,1-9]','')
print(text)

0            pencils sharpened
1            pencils sharpened
2            pencils sharpened
3            pencils sharpened
4            pencils sharpened
                  ...         
419725    zephyr polishing kit
419726    zephyr polishing kit
419727    zephyr polishing kit
419728    zephyr polishing kit
419729    zephyr polishing kit
Name: query, Length: 419730, dtype: object


/tmp/ipykernel_10204/165082349.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('[#,@,&,1-9]','')


**Criando vetores**

In [11]:
import requests
from io import StringIO
#import pandas as pd

In [12]:
#sentences = train_df['query'].tolist()
sentences = text.tolist()

In [13]:
# remove duplicates and NaN
#sentences = [word for word in list(set(sentences)) if type(word) is str]
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [14]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens') 
#paraphrase-multilingual-MiniLM-L12-v2

/home/erick.correia/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(20815, 768)

Tratamento de dados de teste

## IndexFlatL2



In [16]:
import faiss

In [17]:
size = sentence_embeddings.shape[1]
size

768

In [18]:
index = faiss.IndexFlatL2(size)

In [19]:
index.is_trained

True

In [20]:
index.add(sentence_embeddings)

In [21]:
index.ntotal

20815

In [22]:
#Teste
k = 4
xq = model.encode(["envelopes without security tint"])

In [23]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 3470 16750 17587 20170]]
CPU times: user 27.4 ms, sys: 0 ns, total: 27.4 ms
Wall time: 24.9 ms


In [38]:
#train_df['query'].iloc[[19805, 4829, 978, 14630]]
#10620 14711  1269 15458
sentences[3470]

'note cards without envelopes'

In [25]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, size))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [26]:
vecs.shape

(4, 768)

In [27]:
vecs[0][:100]

array([ 0.43119857,  0.91011864,  0.4051747 , -0.06226606,  0.09338088,
       -0.20125297,  1.1626184 , -0.08322101, -0.36759871,  0.20486532,
       -0.93038219, -0.61459619,  0.4171924 ,  0.51536101,  0.88794357,
        0.45378926, -0.57322139,  0.27390629,  0.29525098, -0.19188608,
       -0.24476814,  0.57090127,  0.04908442, -1.19785392, -0.54007924,
       -0.35953718,  0.39977959, -0.30137578, -1.15571201,  0.45229897,
        0.232692  ,  0.55188435,  1.23124671,  0.08548499,  1.41238093,
       -0.40255302, -0.1310015 , -1.07996678, -0.15451427, -0.60550433,
       -0.06073511,  0.32497635,  0.32338566,  0.35931736,  0.13488162,
       -0.05715447,  1.5942353 ,  0.02342471, -0.78752214,  0.07201277,
        0.56621379, -0.50664479,  1.9796375 ,  1.1744287 , -0.15058279,
       -0.67751205,  0.21461996, -0.77990431,  0.93161666, -0.23863807,
        0.13793547,  0.04795064, -0.26176697,  0.12465028, -0.2205309 ,
       -0.32104367,  0.07732798,  0.5417518 , -0.53027701, -1.18

# implementando um indice plano faiss

In [28]:
index_x = faiss.IndexFlatIP(size)

In [29]:
index_x.add(sentence_embeddings)

In [30]:
xq = model.encode(["car paint", "arsoft"])

In [31]:
#Teste
k = 10
D, I = index_x.search(xq, k)
print(I)

[[12983 13585 18938  3294  9513 16141  1653 11524 15252 13054]
 [ 9036  9111  9221 11956  6503 19832  1868   175 15698 17643]]


In [32]:
#train_df['query'].iloc[[10365, 12349, 7469, 7340]]
#12516 15213  7553 12372 18626 15555 11820 20000  5935  4487
sentences[7553]

'es shoes skateboard'

In [33]:
nbits = size*4  # resolution of bucketed vectors
# initialize index and add vectors
index_e = faiss.IndexLSH(size, nbits)
index_e.add(sentence_embeddings)
# and search
D, I = index_e.search(xq, k)
print(I)

[[13585 18938 11584 12983 14988  1653  3294 16141  9513 15252]
 [16613 15698  9111  6503 11043  9036 10212 19436 19188 19832]]


In [34]:
#train_df['query'].iloc[[12349, 7469, 9072, 10365, 4282, 2151, 7340, 50, 11175, 2665 ]]
#[15213  7553  2488 12516  4900 11820 12372 15555 18626  5935
sentences[4900]

'baby boy easter outfit'

In [35]:
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index_h = faiss.IndexHNSWFlat(size, M)
# set efConstruction and efSearch parameters
index_h.hnsw.efConstruction = ef_construction
index_h.hnsw.efSearch = ef_search
# add data to index
index_h.add(xq)

# search as usual
D, I = index.search(xq, k)
print(I)

[[13585 18938 12983 11584 15252  3294  9513 11524  1653 17401]
 [  175  9111 16613 15698 11043  6503 19188 19436 10212 20444]]


In [36]:
sentences[20000]

'bugsy blu ray'